## Process Census Data for Clustering
### Primary Author
Maia Guo

### Description:
This notebook extracts features from the 2019 5-year census data and 2019 travel data for clustering use.

### Inputs:
weekly_trips_20211017.csv

safegraph_open_census_data_2019 (https://drive.google.com/drive/folders/1Thu78vlZ7KnRdXJllzeEErI90lzT6bSc?usp=sharing)


### Output:
cbg_estimated_count_2019.csv

raw_cbg_attr.csv

In [ ]:
!pip install --upgrade pip setuptools wheel
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install geopandas

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
import json
%pylab inline 
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


## Travel Count before COVID

Estimated Count = Visitor Count * Population / Device Count

Groupby CBG: Mean Weekly Estimated Count of POIs

In [ ]:
trips = pd.read_csv('/content/drive/MyDrive/SafeGraph/Data/weekly_trips_20211017.csv')
trips.date_range_start = pd.to_datetime(trips.date_range_start, utc=True)
# filter data in 2019
trips = trips[(trips.date_range_start>=pd.to_datetime('2019-01-01 00:00:00-00:00', utc=True))&(trips.date_range_start<pd.to_datetime('2020-01-01 00:00:00-00:00', utc=True))]
trips.head()

In [ ]:
# median values in over 2019
travel = trips[['visitor_home_cbg_id', 'date_range_start', 'esimated_visitor_count']].groupby(['visitor_home_cbg_id', 'date_range_start'], as_index=False).median()
# median values in each cbg
travel = travel.groupby('visitor_home_cbg_id', as_index=False).median()
travel.head()

In [ ]:
travel.shape

(6338, 2)

In [ ]:
travel.to_csv('/content/drive/MyDrive/SafeGraph/Data/cbg_estimated_count_2019.csv', index=False)

## Census Data Processing

https://www.census.gov/programs-surveys/acs/guidance/which-data-tool/table-ids-explained.html

https://docs.safegraph.com/docs/open-census-data

**01 Population; Age; Sex**

**02 Race** （Latinxs were 1.6 times more likely than whites to report their food access diminished by “a lot.” Asian respondents also reported significantly reduced ability to get food, with 54.8% reporting reduced access by “a lot” or “somewhat.” ）


**11 Household Type**

**15 Educational Attainment**

**17 Poverty Status**

**19 Income** 

**22 Food Stamps/Supplemental Nutrition Assistance Program (SNAP)**

**23 Employment Status; Work Status Last Year** (Current consumption & future expectations)


In [ ]:
census_des = pd.read_csv('/content/drive/MyDrive/SafeGraph/SafeGraph Open Census Data/safegraph_open_census_data_2019/metadata/cbg_field_descriptions.csv') 
census_des.shape

(8120, 15)

In [ ]:
def generate_field_dict(df):
    
    NAME = []
    ID = []
    for i in df.index:
        if str(df.field_level_6[i]) != 'nan':
            NAME.append(df.field_level_6[i])
            ID.append(df.table_id[i])
        elif str(df.field_level_5[i]) != 'nan':
            NAME.append(df.field_level_5[i])
            ID.append(df.table_id[i])
        else:
            NAME.append(df.field_level_3[i])
            ID.append(df.table_id[i])

    field_dict = dict(zip(ID,NAME))
    return field_dict

In [ ]:
def extract_data_columns(field_dict, file_num):
    
    file_path = '/content/drive/MyDrive/SafeGraph/SafeGraph Open Census Data/safegraph_open_census_data_2019/data/cbg_' + file_num + '.csv'
    df = pd.read_csv(file_path)
    df_col = df[['census_block_group'] + list(field_dict.keys())].rename(columns=field_dict).set_index('census_block_group')
    return df_col

In [ ]:
def num_to_percent(df, denominator, column_list):
    
    RENAME = {}
    for c in column_list:
        df[c] = df[c] / df[denominator]
        RENAME[c] = 'Percent ' + c
    df = df.rename(columns=RENAME)

    return df

In [ ]:
# 01 pop, age, gender
census_des[(census_des.table_number.isin(['B01001', 'B01002']))&(census_des.field_level_1=='Estimate')&(census_des.field_level_6.isnull())] 

,table_id,table_number,table_title,table_topics,table_universe,field_level_1,field_level_2,field_level_3,field_level_4,field_level_5,field_level_6,field_level_7,field_level_8,field_level_9,field_level_10
0,B01001e1,B01001,Sex By Age,Age and Sex,Total population,Estimate,SEX BY AGE,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
11,B01001e2,B01001,Sex By Age,Age and Sex,Total population,Estimate,SEX BY AGE,Total population,Total,Male,NaN,NaN,NaN,NaN,NaN
18,B01001e26,B01001,Sex By Age,Age and Sex,Total population,Estimate,SEX BY AGE,Total population,Total,Female,NaN,NaN,NaN,NaN,NaN
122,B01002e1,B01002,Median Age By Sex,Age and Sex,Total population,Estimate,MEDIAN AGE BY SEX,Total population,Median age,Total,NaN,NaN,NaN,NaN,NaN
123,B01002e2,B01002,Median Age By Sex,Age and Sex,Total population,Estimate,MEDIAN AGE BY SEX,Total population,Median age,Male,NaN,NaN,NaN,NaN,NaN
124,B01002e3,B01002,Median Age By Sex,Age and Sex,Total population,Estimate,MEDIAN AGE BY SEX,Total population,Median age,Female,NaN,NaN,NaN,NaN,NaN


In [ ]:
pop_dict = {'B01001e1': 'Total Population',
            'B01001e2': 'Male',
            'B01002e1': 'Median Age'} # 'B01001e26': 'Female',
pop = extract_data_columns(pop_dict, 'b01')
# pop['Gender Ratio'] = pop['Male'] / pop['Female'] # lots of inf
pop = num_to_percent(pop, 'Total Population', ['Male'])
pop.head()

,Total Population,Percent Male,Median Age
census_block_group,,,
10010201001,730,0.401370,43.3
10010201002,1263,0.486144,38.2
10010202001,835,0.470659,38.7
10010202002,1124,0.591637,42.9
10010203001,2774,0.512978,37.3


In [ ]:
# 02 Race 
race_des = census_des[census_des.table_number.isin(['B02%s' % str(i).zfill(3) for i in range(1, 14)])&
           (census_des.field_level_1=='Estimate')&
           (census_des.table_title=='Race')]
generate_field_dict(race_des)

{'B02001e1': 'Total population',
 'B02001e10': 'Two races excluding Some other race and three or more races',
 'B02001e2': 'White alone',
 'B02001e3': 'Black or African American alone',
 'B02001e4': 'American Indian and Alaska Native alone',
 'B02001e5': 'Asian alone',
 'B02001e6': 'Native Hawaiian and Other Pacific Islander alone',
 'B02001e7': 'Some other race alone',
 'B02001e8': 'Two or more races',
 'B02001e9': 'Two races including Some other race'}

In [ ]:
race_dict = generate_field_dict(race_des)
race = extract_data_columns(race_dict, 'b02')
# merge Asian Alone & Two races including Some other race
race['Asian alone & Two races including Some other race'] = race['Asian alone'] + race['Two races including Some other race']
race = race.drop(['Asian alone', 'Two races including Some other race'], axis=1)
race = num_to_percent(race, 'Total population', race.columns[1:])
race = race.iloc[:, 1:]
race.head()

,Percent Two races excluding Some other race and three or more races,Percent White alone,Percent Black or African American alone,Percent American Indian and Alaska Native alone,Percent Native Hawaiian and Other Pacific Islander alone,Percent Some other race alone,Percent Two or more races,Percent Asian alone & Two races including Some other race
census_block_group,,,,,,,,
10010201001,0.078082,0.839726,0.082192,0.0,0.000000,0.000000,0.078082,0.000000
10010201002,0.076801,0.848773,0.072842,0.0,0.000000,0.000000,0.076801,0.001584
10010202001,0.037126,0.330539,0.607186,0.0,0.025150,0.000000,0.037126,0.000000
10010202002,0.022242,0.429715,0.542705,0.0,0.000000,0.005338,0.022242,0.000000
10010203001,0.029200,0.672675,0.200793,0.0,0.001802,0.083994,0.029200,0.011536


In [ ]:
# 11 Household Type
B11 = ['B11005', 'B11006', 'B11007'] # family type & children 'B11003'
household_des = census_des[census_des.table_number.isin(B11)&
           (census_des.field_level_1=='Estimate')&
           (census_des.field_level_6.isnull())&
           (census_des.field_level_5.notnull())]
generate_field_dict(household_des)

{'B11005e11': 'Households with no people under 18 years',
 'B11005e2': 'Households with one or more people under 18 years',
 'B11006e2': 'Households with one or more people 60 years and over',
 'B11006e9': 'Households with no people 60 years and over',
 'B11007e2': 'Households with one or more people 65 years and over',
 'B11007e7': 'Households with no people 65 years and over'}

In [ ]:
household_dict = generate_field_dict(household_des)
household_dict['B11005e1'] = 'Total Households'
household = extract_data_columns(household_dict, 'b11')
household = num_to_percent(household, 'Total Households', household.columns[:-1])
household = household[['Total Households',
                       'Percent Households with one or more people under 18 years', 
                       'Percent Households with one or more people 60 years and over']]
household.head()

,Total Households,Percent Households with one or more people under 18 years,Percent Households with one or more people 60 years and over
census_block_group,,,
10010201001,315,0.285714,0.425397
10010201002,394,0.385787,0.276650
10010202001,318,0.402516,0.490566
10010202002,370,0.170270,0.335135
10010203001,1026,0.416179,0.334308


In [ ]:
# 15 Educational Attainment
B15 = ['B15003', 'B15011']
edu_des = census_des[census_des.table_number.isin(B15)&
           (census_des.field_level_1=='Estimate')&
           (census_des.field_level_6.isnull())]
generate_field_dict(edu_des)

{'B15003e1': 'Population 25 years and over',
 'B15003e10': '6th grade',
 'B15003e11': '7th grade',
 'B15003e12': '8th grade',
 'B15003e13': '9th grade',
 'B15003e14': '10th grade',
 'B15003e15': '11th grade',
 'B15003e16': '12th grade no diploma',
 'B15003e17': 'Regular high school diploma',
 'B15003e18': 'GED or alternative credential',
 'B15003e19': 'Some college less than 1 year',
 'B15003e2': 'No schooling completed',
 'B15003e20': 'Some college 1 or more years no degree',
 'B15003e21': "Associate's degree",
 'B15003e22': "Bachelor's degree",
 'B15003e23': "Master's degree",
 'B15003e24': 'Professional school degree',
 'B15003e25': 'Doctorate degree',
 'B15003e3': 'Nursery school',
 'B15003e4': 'Kindergarten',
 'B15003e5': '1st grade',
 'B15003e6': '2nd grade',
 'B15003e7': '3rd grade',
 'B15003e8': '4th grade',
 'B15003e9': '5th grade',
 'B15011e1': "POPULATION 25 YEARS AND OVER WITH A BACHELOR'S DEGREE OR HIGHER ATTAINMENT",
 'B15011e2': 'Male',
 'B15011e21': 'Female'}

In [ ]:
# Percent over 25 without a high school diploma
edu_dict = generate_field_dict(edu_des)
edu = extract_data_columns(edu_dict, 'b15')
edu.head()

UnderHighSchool = ['6th grade', '7th grade', '8th grade',
                  '9th grade', '10th grade', '11th grade', '12th grade no diploma', 
                  'No schooling completed', '1st grade',
                  '2nd grade', '3rd grade', '4th grade', '5th grade',]
edu['Percent over 25 without a high school diploma'] = edu[UnderHighSchool].sum(axis=1) / edu['Population 25 years and over']
edu['Percent over 25 with a bachelor’s degree or higher'] = edu["POPULATION 25 YEARS AND OVER WITH A BACHELOR'S DEGREE OR HIGHER ATTAINMENT"] / edu['Population 25 years and over']
edu = edu[['Percent over 25 without a high school diploma', 'Percent over 25 with a bachelor’s degree or higher']]
edu.head()

,Percent over 25 without a high school diploma,Percent over 25 with a bachelor’s degree or higher
census_block_group,,
10010201001,0.207843,0.178431
10010201002,0.073801,0.329643
10010202001,0.125230,0.095764
10010202002,0.162791,0.218605
10010203001,0.126368,0.196517


In [ ]:
# 17 Poverty Status

B17 = ['B17017', ] # 'C17002' age ratio 'B17021' pop 
poverty_des = census_des[census_des.table_number.isin(B17)&
           (census_des.field_level_1=='Estimate')&
           (census_des.field_level_6.isnull())]
generate_field_dict(poverty_des)

{'B17017e1': 'Households',
 'B17017e2': 'Income in the past 12 months below poverty level',
 'B17017e31': 'Income in the past 12 months at or above poverty level'}

In [ ]:
poverty_dict = generate_field_dict(poverty_des)
b17 = pd.read_csv('/content/drive/MyDrive/SafeGraph/SafeGraph Open Census Data/safegraph_open_census_data_2019/data/cbg_b17.csv')
poverty = b17[['census_block_group'] + list(poverty_dict.keys())].rename(columns=poverty_dict).set_index('census_block_group')
poverty = num_to_percent(poverty, 'Households', poverty.columns[1:])
poverty = poverty[['Percent Income in the past 12 months below poverty level']]
poverty.columns = ['Percent Households Income in the past 12 months below poverty level']
poverty.head()

,Percent Households Income in the past 12 months below poverty level
census_block_group,
10010201001,0.209524
10010201002,0.134518
10010202001,0.421384
10010202002,0.108108
10010203001,0.099415


In [ ]:
# 19 Income
census_des[(census_des.table_number == 'B19013')&(census_des.field_level_1=='Estimate')]
b19 = pd.read_csv('/content/drive/MyDrive/SafeGraph/SafeGraph Open Census Data/safegraph_open_census_data_2019/data/cbg_b19.csv')
income = b19[['census_block_group', 'B19013e1']].rename(columns={'B19013e1': 'Median Household Income'}).set_index('census_block_group')
income.head()

,Median Household Income
census_block_group,
10010201001,35703.0
10010201002,79000.0
10010202001,26500.0
10010202002,51042.0
10010203001,59048.0


In [ ]:
# 22 Food Stamps/Supplemental Nutrition Assistance Program (SNAP)
snap_des = census_des[census_des.table_number.isin(['B22%s' % str(i).zfill(3) for i in range(1, 20)])&
           (census_des.field_level_1=='Estimate')]
generate_field_dict(snap_des)

{'B22010e1': 'Households',
 'B22010e2': 'Household received Food Stamps/SNAP in the past 12 months',
 'B22010e3': 'Households with 1 or more persons with a disability',
 'B22010e4': 'Households with no persons with a disability',
 'B22010e5': 'Household did not receive Food Stamps/SNAP in the past 12 months',
 'B22010e6': 'Households with 1 or more persons with a disability',
 'B22010e7': 'Households with no persons with a disability'}

In [ ]:
snap_dict = {'B22010e1': 'Households',
             'B22010e2': 'Household received Food Stamps/SNAP',
             'B22010e3': 'Households received Food Stamps/SNAP have disability',
             'B22010e5': 'Household did not receive Food Stamps/SNAP',
             'B22010e6': 'Households did not received Food Stamps/SNAP with disability'}
b22 = pd.read_csv('/content/drive/MyDrive/SafeGraph/SafeGraph Open Census Data/safegraph_open_census_data_2019/data/cbg_b22.csv')
snap = b22[['census_block_group'] + list(snap_dict.keys())].rename(columns=snap_dict).set_index('census_block_group')
# snap['Percent Households with disability received Food Stamps/SNAP'] = snap.iloc[:, 2] / (snap.iloc[:, 2] + snap.iloc[:, 4])
snap = num_to_percent(snap, 'Households', snap.columns[1:5]).drop('Households', axis=1)
snap.iloc[:, 1] = snap.iloc[:, 1] / snap.iloc[:, 0]
snap = snap.drop(['Percent Household did not receive Food Stamps/SNAP', 
                  'Percent Households did not received Food Stamps/SNAP with disability'], axis=1)
snap.head()

,Percent Household received Food Stamps/SNAP,Percent Households received Food Stamps/SNAP have disability
census_block_group,,
10010201001,0.196825,0.516129
10010201002,0.157360,0.483871
10010202001,0.383648,0.196721
10010202002,0.140541,0.307692
10010203001,0.117934,0.793388


In [ ]:
# 23 Employment
# census_des.table_number.unique()
census_des[census_des.table_id.isin(['B23025e4', 'B23025e5', 'B23025e3'])]

,table_id,table_number,table_title,table_topics,table_universe,field_level_1,field_level_2,field_level_3,field_level_4,field_level_5,field_level_6,field_level_7,field_level_8,field_level_9,field_level_10
4164,B23025e3,B23025,Employment Status For The Population 16 Years ...,Employment and Labor Force Status,Population 16 years and over,Estimate,EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS ...,Population 16 years and over,Total,In labor force,Civilian labor force,NaN,NaN,NaN,NaN
4165,B23025e4,B23025,Employment Status For The Population 16 Years ...,Employment and Labor Force Status,Population 16 years and over,Estimate,EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS ...,Population 16 years and over,Total,In labor force,Civilian labor force,Employed,NaN,NaN,NaN
4166,B23025e5,B23025,Employment Status For The Population 16 Years ...,Employment and Labor Force Status,Population 16 years and over,Estimate,EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS ...,Population 16 years and over,Total,In labor force,Civilian labor force,Unemployed,NaN,NaN,NaN


In [ ]:
c23 = pd.read_csv('/content/drive/MyDrive/SafeGraph/SafeGraph Open Census Data/safegraph_open_census_data_2019/data/cbg_b23.csv')
employment = c23[['census_block_group', 'B23025e4', 'B23025e3']].rename(
    columns={'B23025e4': 'Employed', 'B23025e3': 'Civilian labor force'}).set_index('census_block_group') # 'B23025e5', , 'B23025e5': 'Unemployed'
employment = num_to_percent(employment, 'Civilian labor force', ['Employed']) # , 'Unemployed'
employment.head()

,Percent Employed,Civilian labor force
census_block_group,,
10010201001,0.971875,320
10010201002,0.971212,660
10010202001,1.000000,253
10010202002,0.934394,503
10010203001,0.961640,1512


In [ ]:
# merge the demographic features 
census_df = pd.concat([pop, race, household, edu, poverty, income, snap, employment], axis=1)
cbg = gpd.read_file('/content/drive/MyDrive/SafeGraph/Data/nyc_cbgs.geojson')
census_df = census_df[census_df.index.isin(cbg.CensusBlockGroup)]#.fillna(0)
census_df = num_to_percent(census_df, 'Total Population', ['Civilian labor force'])
census_df.head()

,Total Population,Percent Male,Median Age,Percent Two races excluding Some other race and three or more races,Percent White alone,Percent Black or African American alone,Percent American Indian and Alaska Native alone,Percent Native Hawaiian and Other Pacific Islander alone,Percent Some other race alone,Percent Two or more races,Percent Asian alone & Two races including Some other race,Total Households,Percent Households with one or more people under 18 years,Percent Households with one or more people 60 years and over,Percent over 25 without a high school diploma,Percent over 25 with a bachelor’s degree or higher,Percent Households Income in the past 12 months below poverty level,Median Household Income,Percent Household received Food Stamps/SNAP,Percent Households received Food Stamps/SNAP have disability,Percent Employed,Percent Civilian labor force
census_block_group,,,,,,,,,,,,,,,,,,,,,,
360050001000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360050001001,6864,0.924388,35.0,0.015006,0.325321,0.557110,0.002622,0.0,0.073572,0.015006,0.026369,0,NaN,NaN,0.515975,0.014873,NaN,NaN,NaN,NaN,NaN,0.000000
360050002000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360050002001,1744,0.444381,40.2,0.014908,0.460436,0.331422,0.000000,0.0,0.193234,0.014908,0.000000,460,0.423913,0.541304,0.246892,0.277975,0.326087,41968.0,0.334783,0.357143,0.762500,0.366972
360050002002,1569,0.527725,28.9,0.013384,0.368388,0.291906,0.000000,0.0,0.281071,0.058636,0.045252,477,0.421384,0.425577,0.243154,0.303395,0.144654,48523.0,0.190776,0.637363,0.860104,0.369025


In [ ]:
census_df.shape

(6493, 22)

In [ ]:
census_df.to_csv('/content/drive/MyDrive/SafeGraph/Data/raw_cbg_attr.csv', index=True)